In [ ]:
# use /home/vr/venv_cluster
# Test out SHAP deepExplainer for visualise saliency regions
# reference: 
# https://shap-lrjball.readthedocs.io/en/latest/example_notebooks/deep_explainer/PyTorch%20Deep%20Explainer%20MNIST%20example.html
# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torchvision.models import resnet50, resnet18, densenet121, vgg11_bn, alexnet

import numpy as np
import shap

%env CUDA_VISIBLE_DEVICES=1,2

In [ ]:
batch_size = 128
num_epochs = 2
device = torch.device('cuda')
num_classes = 10 # to change depend on which dataset used


In [ ]:
def get_backbone(backbone, castrate=True):
    backbone = eval(f"{backbone}()")

    if castrate:
        backbone.output_dim = backbone.fc.in_features
        backbone.fc = torch.nn.Identity() # no ops just forwarding the input given to it

    return backbone, backbone.output_dim

# during inferencing, freezing layers or feature extract, grad is not needed
def set_parameter_requires_grad_false(model):
    for param in model.parameters():
        param.requires_grad = False

In [ ]:
# pretrained models: resnet50, resnet18, densenet121, vgg11_bn, alexnet
model = resnet18(pretrained=True)

num_ftrs = model.fc.in_features # resnet
#num_ftrs = model.classifier.in_features # densenet
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, num_classes)

#num_ftrs = model.classifier[6].in_features # vgg11_bn, alexnet
#model.classifier[6] = nn.Linear(num_ftrs,num_classes)


In [ ]:
# Resnet50 pretrained model by Simsiam
model_dir ='/home/vr/simsiam_cache/simsiam-resnet50-augs-fullset-hand_0327170940.pth'
backbone ='resnet50'  # 'resnet18_cifar_variant1' 'resnet50'
model, dim_feature_last = get_backbone(backbone)
save_dict = torch.load(model_dir, map_location=device)

# set the strict argument to **False** in the load_state_dict() function to ignore non-matching keys
msg = model.load_state_dict({k[9:]:v for k, v in save_dict['state_dict'].items() if k.startswith('backbone.')}, strict=True)
print(msg)

set_parameter_requires_grad_false(model)
# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
# model.fc = nn.Sequential(
#    nn.Linear(dim_feature_last, 50),
#    nn.ReLU(),
#    nn.Dropout(),
#    nn.Linear(50, num_classes),
#    nn.Softmax(dim=1),
#)

model.fc = nn.Sequential(
    nn.Linear(dim_feature_last, num_classes)
)


In [ ]:
#print({k[9:]:v for k, v in save_dict['state_dict'].items() if k.startswith('backbone.')})
#print(save_dict)
x = torch.randn(2, 2)
print((x.unsqueeze_(0)).repeat(3,1,1))

In [ ]:
criterion = nn.CrossEntropyLoss()
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        #print('data: {}, output: {}, output.log(): {}, target: {}'.format(data.shape, output.shape, (output.log()).shape,target.shape))
        
        #loss = F.nll_loss(output.log(), target)
        loss = criterion(output, target)
        #loss = criterion(output.log(), target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += F.nll_loss(output.log(), target).item() # sum up batch loss
            #test_loss += criterion(output.log(), target).item()
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            #print('gt:{}, pred:{}', target.view_as(pred), pred)

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
# MINST 
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       #transforms.Resize(224),
                       transforms.ToTensor(),
                       #transforms.Lambda(lambda x: x.repeat(3,1,1))
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
                       #transforms.Resize(224),
                       transforms.ToTensor(),
                       #transforms.Lambda(lambda x: x.repeat(3,1,1))
                   ])),
    batch_size=batch_size, shuffle=True)

In [ ]:
# CIFAR 10
# Image preprocessing modules
transform = transforms.Compose([
    #transforms.Resize(224),
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    #transforms.RandomCrop(224),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())
                                            #transform=transforms.Compose([transforms.Resize(224),transforms.ToTensor()]))

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [ ]:
print(batch_size)

In [ ]:
model = model.to(device)
# Optimize only the classifier
#optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

In [ ]:
# Deepexplainer NOTE: ResNet50 report dimension error
# since shuffle=True, this is a random sample of test data
batch = next(iter(test_loader))
images, _ = batch

background = images[:100]
test_images = images[100:103]

print(test_images.shape)

model = model.to('cpu')
e = shap.DeepExplainer(model, background)
#print(e.supports_model(model))
shap_values = e.shap_values(test_images)

In [ ]:
shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
test_numpy = np.swapaxes(np.swapaxes(test_images.numpy(), 1, -1), 1, 2)

In [ ]:
# plot the feature attributions
shap.image_plot(shap_numpy, test_numpy)

In [ ]:
# Gradient exmplainer
# refer to example https://shap-lrjball.readthedocs.io/en/latest/example_notebooks/gradient_explainer/Explain%20an%20Intermediate%20Layer%20of%20VGG16%20on%20ImageNet%20%28PyTorch%29.html

# print(model.features.denseblock2.denselayer12.conv2) # Densenet

batch = next(iter(test_loader))
images, _ = batch
background = images[:100]
test_images = images[100:102]

to_show = np.swapaxes(np.swapaxes(test_images.numpy(), 1, -1), 1, 2)
#print(to_show.shape)

model = model.to('cpu')

e = shap.GradientExplainer((model, model.layer3[0]), background) # resnet
#e = shap.GradientExplainer((model, model.features[7]), background) #vgg
shap_values,indexes = e.shap_values(test_images, ranked_outputs=2, nsamples=200)
# plot the explanations
shap_values = [np.swapaxes(np.swapaxes(s, 2, 3), 1, -1) for s in shap_values]

shap.image_plot(shap_values, to_show)

In [ ]:
#print(model)